In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import xgboost
from sklearn import metrics
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
df_train = pd.read_csv('../input/train.csv')
df_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
y = df_train.label.values
X = df_train.drop('label', axis=1).values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(37800, 784) (4200, 784) (37800,) (4200,)


In [4]:


dtrain = xgboost.DMatrix(X_train, label=y_train)
dval = xgboost.DMatrix(X_val, label=y_val)

params = {
    'max_depth': 5,                 # the maximum depth of each tree
    'eta': 0.5,                     # the training step for each iteration
    'eta_decay': 0.9,
    'min_eta': 0.05,
    'silent': 0,                    # logging mode - quiet
    'objective': 'multi:softmax',   # multiclass classification using the softmax objective
    'num_class': 10,                 # the number of classes that exist in this datset
    'early_stopping_rounds': 10,
    'n_estimators': 5000
}  

def eval_acc(preds, dtrain):
    labels = dtrain.get_label()
    return 'acc', metrics.accuracy_score(labels, preds)

results = {}
model = xgboost.train(params, dtrain, num_boost_round=100, evals=[(dval, 'val')], evals_result=results, feval=eval_acc)

[0]	val-merror:0.177143	val-acc:0.822857
[1]	val-merror:0.127857	val-acc:0.872143
[2]	val-merror:0.11119	val-acc:0.88881
[3]	val-merror:0.099286	val-acc:0.900714
[4]	val-merror:0.09	val-acc:0.91
[5]	val-merror:0.081429	val-acc:0.918571
[6]	val-merror:0.076429	val-acc:0.923571
[7]	val-merror:0.073095	val-acc:0.926905
[8]	val-merror:0.066429	val-acc:0.933571
[9]	val-merror:0.062143	val-acc:0.937857
[10]	val-merror:0.057143	val-acc:0.942857
[11]	val-merror:0.055714	val-acc:0.944286
[12]	val-merror:0.053571	val-acc:0.946429
[13]	val-merror:0.047857	val-acc:0.952143
[14]	val-merror:0.04619	val-acc:0.95381
[15]	val-merror:0.043333	val-acc:0.956667
[16]	val-merror:0.043571	val-acc:0.956429
[17]	val-merror:0.043333	val-acc:0.956667
[18]	val-merror:0.042143	val-acc:0.957857
[19]	val-merror:0.040714	val-acc:0.959286
[20]	val-merror:0.038571	val-acc:0.961429
[21]	val-merror:0.037619	val-acc:0.962381
[22]	val-merror:0.036905	val-acc:0.963095
[23]	val-merror:0.036667	val-acc:0.963333
[24]	val-merro

In [5]:
df_test = pd.read_csv('../input/test.csv')
df_test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
x_test = df_test.values
dtest = xgboost.DMatrix(x_test)

In [7]:
preds = model.predict(dtest)
print(preds.shape)
preds[:2]

(28000,)


array([2., 0.], dtype=float32)

In [8]:
df_sub = pd.read_csv('../input/sample_submission.csv')
df_sub.head()

,ImageId,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [9]:
df_sub.Label = preds.astype(np.int32)
df_sub.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [10]:
df_sub.to_csv('submission.csv', index=False)